In [1]:
from pymongo import MongoClient
import re

mc = MongoClient('192.168.194.171', port=27017)
db = mc.db_restT

In [113]:
tweets = db.restT.find(
    {
        "retweeted_status.lang" : "en",
        "retweeted_status.user.id_str" : "1130534810"
    }
).limit(1)
for tweet in tweets:
    print tweet['user']['id_str']
    print tweet['retweeted_status']['user']['id_str']
    print tweet['retweeted_status']['user']['name']
    print tweet['retweeted_status']['user']['screen_name']
    print tweet['retweeted_status']['retweet_count']
    print tweet['retweeted_status']['id_str']
    print tweet['retweeted_status']['text']
    print '---------------------------------------------------------------'

201871438
1130534810
NBA On Def Pen
NBAOnDefPen
8
611345721980256256
The Curry's #NBAFinals2015 http://t.co/ZVmvQI7zdD
---------------------------------------------------------------


In [109]:
retweets = {}
tweets = db.restT.find(
    {
        "retweeted_status.lang" : "en",
    }
).limit(1)
for tweet in tweets:
    retweets['author_id'] = tweet['retweeted_status']['user']['id_str']
    retweets['text']      = tweet['retweeted_status']['text']
    retweets['id_str']    = tweet['retweeted_status']['id_str']
    retweets['retweeter'] = {'user' : tweet['user']['id_str']}
    print str(retweets)
#    db.retweets.insert(retweet)

{'text': u'The wait is over \u2013 Your Golden State #Warriors are #NBA CHAMPIONS!!! #GSW #NBAFinals http://t.co/3xxxrnJbJN', 'author_id': u'26270913', 'retweeter': {'user': u'1595719178'}, 'id_str': u'611020181310025729'}


In [95]:
retweets = db.retweet.find()
for retweet in retweets:
    print retweet

{u'contributors': None, u'truncated': False, u'text': u'#LeBronsDick live at the #NBAFinals2015 \U0001f602 http://t.co/Uy4QSUMwfJ', u'is_quote_status': False, u'in_reply_to_status_id': None, u'id': 609203651282882560L, u'favorite_count': 88, u'source': u'<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', u'retweeted': False, u'coordinates': None, u'entities': {u'symbols': [], u'user_mentions': [], u'hashtags': [{u'indices': [0, 12], u'text': u'LeBronsDick'}, {u'indices': [25, 39], u'text': u'NBAFinals2015'}], u'urls': [], u'media': [{u'expanded_url': u'http://twitter.com/FashionistaKika/status/609203651282882560/photo/1', u'display_url': u'pic.twitter.com/Uy4QSUMwfJ', u'url': u'http://t.co/Uy4QSUMwfJ', u'media_url_https': u'https://pbs.twimg.com/media/CHRTfLmUAAAWwrr.jpg', u'id_str': u'609203644756393984', u'sizes': {u'small': {u'h': 396, u'w': 340, u'resize': u'fit'}, u'large': {u'h': 670, u'w': 574, u'resize': u'fit'}, u'medium': {u'h': 670, u'w': 57

In [136]:
stuff = db.restT.aggregate( 
    [
        { "$match" : { "retweeted_status.lang" : "en"}},
        { "$group" : { "_id" : {"userid" : "$retweeted_status.user.id_str", 
                                "username" : "$retweeted_status.user.name", 
                                "loc" : "$retweeted_status.user.location",
                                "text" : "$retweeted_status.text"},
                       "totRetweets" : {"$sum" : 1 }}},
        { "$sort" : {"totRetweets" : -1}},
        { "$limit" : 30}
    ])

In [137]:
for things in stuff:
    print things


{u'_id': {u'username': u'Golden St. Warriors', u'loc': u'Oakland, CA', u'userid': u'26270913', u'text': u'The wait is over \u2013 Your Golden State #Warriors are #NBA CHAMPIONS!!! #GSW #NBAFinals http://t.co/3xxxrnJbJN'}, u'totRetweets': 27705}
{u'_id': {u'username': u'Golden St. Warriors', u'loc': u'Oakland, CA', u'userid': u'26270913', u'text': u'The #Warriors have won their first NBA Championship in 40 years after defeating the Cavs 105-97 in Game 6. http://t.co/kLe2TzALQu'}, u'totRetweets': 3607}
{u'_id': {u'username': u'Golden St. Warriors', u'loc': u'Oakland, CA', u'userid': u'26270913', u'text': u'#Warriors close 1Q out on 8-0 run to take a 13-point lead into the startof 2Q. #StrengthInNumbers #GSW #NBAFinals http://t.co/0fKJzzwiHa'}, u'totRetweets': 1459}
{u'_id': {u'username': u'Golden St. Warriors', u'loc': u'Oakland, CA', u'userid': u'26270913', u'text': u'#Warriors hit 6 treys in first half and take a 12-point lead into the half. #NBAFinals #DubNation #StrengthInNumbers htt

In [106]:
print stuff.next()

{u'_id': u'27189557', u'totRetweets': 6154}
